# Conducting Automated/Autonomous Experiments with MADSci

This aims to teach you how to define and run automated/autonomous Experimental Campaigns using one or more MADSci Workcells.

## Goals

After completing this notebook, you should understand

1. How to leverage a MADSci Experiment Manager via the Experiment Application
2. How to define and use MADSci `ExperimentApplication`s to manage data, resources, logging and workflows for you scientific applications. 


## What is an Experiment Application?

Once we have established the managers that we will use in our lab, we need an efficient way to interact with and organize the workflows, resources and datapoints that we will use to actually do science. To do this, we need to create an Experiment Application. An Experiment Application is a Python class that inherets from the abstract `ExperimentApplication` class defined in `madsci.client`. It contains an `ExperimentClient`, which communicates with the Experiment Manager. 

In [ ]:
# Install dependencies
%pip install madsci.client

In [ ]:
from madsci.client.experiment_application import ExperimentApplication, ExperimentDesign


# Here is the example start of an Experiment Application
class ExampleExperimentApplication(ExperimentApplication):
    url = "http://localhost:8002/"
    experiment_design = ExperimentDesign(
        experiment_name="Example Experiment",
        experiment_description="An Example Experiment",
    )


experiment_application = ExampleExperimentApplication()
print(experiment_application.experiment_design)

An Experiment Application contains an `ExperimentDesign` object. This can be as simple as the name and description of the experiment, but can also contain checks to ensure that the resources of the lab are in the correct state to begin a run of the experiment, though this functionality is still being tested and refined


By inheriting from `ExperimentApplication`, we gain access to several methods that are useful in organizing our experiments. The first one is `start_experiment`.

In [ ]:
import datetime

experiment_application = ExampleExperimentApplication()

experiment_application.start_experiment_run(
    run_name="first_example_" + str(datetime.datetime.now()),
    run_description="a run of my example experiment",
)

As we can see, this has started a record of our experiment with a specific, timestamped name and a unique id. 
Once our experiment is finished, we can call `end_experiment_run` to show that we have completed this specific experiment. 

In [ ]:
experiment_application.end_experiment()

We can also use it as a context manager!

In [ ]:
from madsci.common.types.workflow_types import WorkflowDefinition

workflow = WorkflowDefinition(
    name="Example Workflow",
    steps=[
        {
            "name": "Run Liquidhandler",
            "description": "Run the Liquidhandler",
            "node": "liquidhandler_1",
            "action": "run_command",
            "args": {"command": "first_command"},
        }
    ],
)

with experiment_application.manage_experiment(
    run_name="first_example_" + str(datetime.datetime.now()),
    run_description="a run of my example experiment",
):
    experiment_application.workcell_client.start_workflow(workflow)

### What can we do with an Experiment application?

#### We can run workflows!
Workflows are a list of ordered steps for different robots that are managed by the workcell manager. Each step contains a specific action for a specific node to run, along with the necessary arguments for that action to run.


In [ ]:
from madsci.common.types.resource_types import Asset
from madsci.common.types.workflow_types import WorkflowDefinition
from madsci.common.types.step_types import StepDefinition
from pathlib import Path

with open(Path("./protocol.py").resolve(), "w") as f:
    f.write("this is a protocol")

workflow_2 = WorkflowDefinition(
    name="Example Workflow 2",
    steps=[
        StepDefinition(
            name="Open Liquidhandler",
            description="Open the Liquidhandler",
            node="liquidhandler_1",
            action="run_command",
            args={"command": "open"},
        ),
        StepDefinition(
            name="Run Liquidhandler Protocol",
            description="Run the Liquidhandler",
            node="liquidhandler_1",
            action="run_protocol",
            files={"protocol": str(Path("./protocol.py").resolve())},
        ),
        StepDefinition(
            name="Transfer from liquid handler",
            description="Transfer an asset from the liquid handler to the plate reader",
            node="robotarm_1",
            action="transfer",
            locations={
                "source": "location_1",
                "target": "location_2",
            },
        ),
    ],
)

asset = experiment_application.resource_client.add_resource(
    Asset(resource_name="my_asset", resource_type="well_plate")
)  # Define a plate
try:
    experiment_application.resource_client.pop(
        "01JR8F3KF3BPZG1BEDK9DKXNWH"
    )  # clear the resource for location_1
except Exception:
    pass
experiment_application.resource_client.push(
    "01JR8F3KF3BPZG1BEDK9DKXNWH", asset
)  # Push to the resource for location_1
try:
    experiment_application.resource_client.pop(
        "01JR8F3KN3MA49C0HW0H727STM"
    )  # Push to the resource for location_1
except Exception:
    pass
with experiment_application.manage_experiment(
    run_name="first_example_" + str(datetime.datetime.now()),
    run_description="a run of my example experiment",
):
    experiment_application.workcell_client.start_workflow(workflow_2)

We can put breakpoints in our experiment that allow us to pause our experiment execution as we see fit. This allows us to pause the experiments execution remotely, in case some conditions arise that require us to pause execution of the workflow. You will have to stop the execution of the below cell, as it will pause itself and get stuck 

In [ ]:
with experiment_application.manage_experiment(
    run_name="first_example_" + str(datetime.datetime.now()),
    run_description="a run of my example experiment",
):
    experiment_application.workcell_client.start_workflow(workflow)
    experiment_application.experiment_client.pause_experiment(
        experiment_application.experiment.experiment_id
    )
    experiment_application.check_experiment_status()
    experiment_application.workcell_client.start_workflow(workflow)

Eventually, pausing the experiment will pause all running workflows owned by the experiment, and even pause the node executions if the devices are capable of being paused, but we have not implemented this yet. 

We are also able to specify a set of conditions regarding resources, to be met before the experiment is allowed to proceed. These are detailed in condition types, but a few examples are shown below.

In [ ]:
from madsci.common.types.condition_types import (
    ResourceInLocationCondition,
    ResourceChildFieldCheckCondition,
)

experiment_design = ExperimentDesign(
    experiment_name="Example Experiment",
    experiment_description="An Example Experiment",
    resource_conditions=[
        ResourceInLocationCondition(
            condition_name="Check that slot_1 has a plate", location_name="location_1"
        ),
        ResourceChildFieldCheckCondition(
            condition_name="Check that the object in slot_1 is a well plate",
            resource_name="slot_1",
            field="resource_type",
            operator="is_equal_to",
            target_value="well_plate",
        ),
    ],
)
experiment_application.experiment_design = experiment_design
with experiment_application.manage_experiment(
    run_name="first_example_" + str(datetime.datetime.now()),
    run_description="a run of my example experiment",
):
    experiment_application.workcell_client.start_workflow(workflow)

Alright, now that we have all of the basics, we are going to write a simple example experiment that shows how we can complete the loop using an Experiment Application